# 패키지 import

In [8]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# Setting, Crawler 클래스 정의

In [41]:
class Setting():
    def __init__(self):
        self.key = self._read_key()
        self.fields = self._read_fields()
        self.area_nms = self._read_area_113()
        
    # config 폴더 내 OPEN_API_KEY.txt에 작성해 둔 OPEN API 인증키를 받아오는 함수
    def _read_key(self):
        try:
            with open('./config/OPEN_API_KEY.txt') as file:
                key = file.read()
            return key
        except:
            raise Exception("OPEN_API_KEY.txt 읽기 오류")
            
    # config 폴더 내 FIELDS.txt에 작성해 둔 필드 값들을 받아오는 함수
    # 해당 필드들은 API 요청으로 받아온 데이터에서 최종적으로 남길 필드들을 의미합니다.
    def _read_fields(self):
        try:
            with open('./config/FIELDS.txt') as file:
                fields = file.read().split()
            return fields
        except:
            raise Exception("FIELDS.txt 읽기 오류")
    
    # data 폴더 내 서울시 주요 113장소명 목록(코드포함).xlsx에서 지역명들을 받아오는 함수
    # 해당 지역명들은 API 호출 시 각각 url 값에 넣어주어야 해서 반드시 필요합니다.
    def _read_area_113(self):
        try:
            area_113 = pd.read_excel('./data/서울시 주요 113장소명 목록(코드포함).xlsx')
            area_nms = area_113['AREA_NM'].to_list()
            return area_nms
        except:
            raise Exception("서울시 주요 113장소명 목록(코드포함).xlsx 읽기 오류")

In [51]:
class Crawler(Setting):
    def __init__(self):
        super().__init__()
    
    # 특정 지역 1개에 대해서만 API 응답 데이터를 받아와서 반환하는 함수
    def get_data_from_api(self, area_nm):
        url = 'http://openapi.seoul.go.kr:8088/' + self.key + '/xml/citydata/1/5/' + area_nm

        response = requests.get(url)
        root = ET.fromstring(response.content)
        result_data = self._get_values(root, self.fields)
        
        return result_data
    
    # 113개 지역 모두에 대해서 API 응답 데이터를 받아와서 리스트로 담아 반환하는 함수
    def get_datas_from_api(self):
        result_data_list = []
        
        for area_nm in self.area_nms:
            result_data = self.get_data_from_api(area_nm)
            result_data_list.append(result_data)
        
        return result_data_list

    # xml 형식 데이터에서 특정 필드의 값만을 찾아서 그 값을 반환하는 함수
    def _get_value(self, root, field):
        for neighbor in root.iter(field):
            return neighbor.text

    # xml 형식 데이터에서 원하는 필드들의 값을 모두 찾아서 dict 형태로 반환하는 함수    
    def _get_values(self, root, fields):
        result_dict = {}
        
        for field in fields:
            result_dict[field] = self._get_value(root, field)
            
        return result_dict

# 실행 예제

In [ ]:
# Crawler 객체 생성
my_crawler = Crawler()

In [45]:
# 113개 지역 모두에 대해서 API 응답 데이터를 받아와서 리스트로 담아 반환하는 예제
result_data_list = my_crawler.get_datas_from_api()
print(result_data_list)

c:\Users\TwinsMin\anaconda3\envs\hp_2\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [49]:
# 특정 지역 1개에 대해서만 API 응답 데이터를 받아와서 반환하는 예제
result_data = my_crawler.get_data_from_api('강남역')
print(result_data)

{'AREA_NM': '강남역', 'AREA_CD': 'POI014', 'LIVE_PPLTN_STTS': None, 'AREA_CONGEST_LVL': '여유', 'PPLTN_TIME': '2023-11-07 02:40', 'WEATHER_STTS': None, 'TEMP': '6.4', 'PM10': '21', 'LAT': None, 'LNG': None, 'ROAD_ADDR': None}
